# Tech Challenge Fase 3 - Análise PNAD-COVID19 IBGE
Data Analytics & Engineering

## 1.Introdução e Arquitetura
Este projeto visa analisar o comportamento da população brasileira durante a pandemia de COVID-19, integrando dados clínicos, demográficos e socioeconômicos. A arquitetura da solução baseia-se em um pipeline de dados utilizando Spark para o processamento de grandes volumes (ETL) e armazenamento em nuvem para posterior análise em ferramentas de Business Intelligence.

### Justificativa da Seleção de Dados

Para esta análise, foram selecionadas 20 variáveis que cruzam três pilares fundamentais para o planejamento hospitalar:

**Características Clínicas:** Sintomatologia (Febre, Tosse, Dificuldade Respiratória, Perda de Olfato/Paladar e Dor de Cabeça) para definição de protocolos de triagem.

**Características da População e Comportamento:** Identificação de grupos de risco por idade/gênero e análise de busca por atendimento em unidades públicas vs. privadas.

**Características Econômicas:** Impacto da renda e do auxílio emergencial na capacidade de isolamento e acesso à saúde.

## 2.Configuração do Ambiente e Ingestão
Nesta etapa, preparamos o ambiente Spark e realizamos a leitura dos microdados referentes aos meses de Setembro, Outubro e Novembro de 2020.

In [ ]:
# Instalação e Configuração
!pip install pyspark -q
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

spark = SparkSession.builder.appName("PNAD_COVID_Analysis").getOrCreate()

# Carga de Microdados (Setembro, Outubro, Novembro)
files = ['PNAD_COVID_092020.csv', 'PNAD_COVID_102020.csv', 'PNAD_COVID_112020.csv']
df_raw = spark.read.csv(files, header=True, inferSchema=True)

print(f"Registros consolidados: {df_raw.count()}")

## 3.Transformação e Curadoria de Dados (ETL)
Seleção das 20 variáveis críticas conforme o planejamento hospitalar e tradução dos códigos numéricos do IBGE para categorias descritivas, facilitando o Storytelling.

In [ ]:
# Transformação e mapeamento das 20 variáveis selecionadas
df_final = df_raw.select(
    col("V1013").alias("mes_referencia"),
    col("UF").alias("unidade_federativa"),
    col("A002").alias("idade"),
    when(col("A003") == 1, "Masculino").otherwise("Feminino").alias("genero"),
    when(col("V1022") == 1, "Urbana").otherwise("Rural").alias("tipo_area"),

    # Sintomatologia (Clínicos)
    when(col("B0011") == 1, "Sim").otherwise("Não").alias("sintoma_febre"),
    when(col("B0012") == 1, "Sim").otherwise("Não").alias("sintoma_tosse"),
    when(col("B0014") == 1, "Sim").otherwise("Não").alias("sintoma_dif_respiratoria"),
    when(col("B0016") == 1, "Sim").otherwise("Não").alias("sintoma_dor_cabeca"),
    when(col("B00111") == 1, "Sim").otherwise("Não").alias("sintoma_perda_olfato_paladar"),

    # Comportamento e Desfecho
    when(col("B002") == 1, "Sim").otherwise("Não").alias("buscou_atendimento"),
    when(col("B006") == 1, "Sim").otherwise("Não").alias("foi_internado"),
    when(col("B007") == 1, "Sim").otherwise("Não").alias("realizou_teste"),
    when(col("B008") == 1, "Positivo").when(col("B008") == 2, "Negativo").otherwise("Inconclusivo").alias("resultado_teste"),
    when(col("B011") == 1, "Rigoroso").when(col("B011") == 2, "Moderado").otherwise("Nenhum").alias("distanciamento_social"),
    when(col("B0042") == 1, "Hospital_SUS").when(col("B0044") == 1, "Hospital_Privado").otherwise("Outros").alias("local_atendimento"),

    # Indicadores Socioeconômicos
    col("C007").alias("tipo_cargo"),
    col("C01012").alias("faixa_rendimento"),
    when(col("D0051") == 1, "Sim").otherwise("Não").alias("recebeu_auxilio"),
    col("D0053").alias("valor_auxilio")
)

# Comandos para visualizar o resultado no Notebook
df_final.show(n=10, truncate=False) # Exibe as primeiras 10 linhas
print(f"Estrutura final: {len(df_final.columns)} colunas processadas.")

## 4.Exportação e Encerramento do Pipeline

In [ ]:
# Exportação do dataset curado para uso em ferramentas de BI (Looker/Power BI)
df_final.coalesce(1).write.csv("pnad_covid_final_curated", header=True, mode="overwrite")

print("Pipeline finalizado. Arquivo 'pnad_covid_final_curated' disponível para download.")

## 5.Breve Análise e Plano de Ação Estratégico

**Principais Descobertas da Base de Dados:**

**Pressão no Sistema:** A correlação entre os sintomas de dificuldade respiratória e a taxa de internação efetiva serve como um indicador antecedente para a ocupação de leitos críticos.

**Perfil Socioeconômico:** A base revela a dependência do auxílio emergencial para a manutenção do isolamento social em estratos de baixa renda, impactando diretamente na velocidade de disseminação regional.


**Ações Recomendadas para o Hospital:**

**Monitoramento de Estoque Antecipado:** Implementar um "Early Warning System" baseado no aumento de relatos de sintomas leves (Febre e Tosse) para prever a necessidade de oxigênio e EPIs em 14 dias.

**Expansão de Telemedicina:** Direcionar recursos de triagem remota para áreas com alto índice de sintomas reportados, visando reduzir a superlotação física do pronto-socorro.

**Gestão de Escala de Profissionais:** Ajustar a escala de plantonistas conforme o comportamento sazonal de picos de internação observados nos microdados de Setembro a Novembro.

**Campanhas de Prevenção Direcionadas:** Focar ações educativas em regiões (UF) e áreas (Urbana/Rural) que apresentaram os menores índices de distanciamento social rigoroso.